In [ ]:
import(
    "crypto/rand"
    "crypto/rsa"
    "crypto/sha256"
    "fmt"
)

In [ ]:
secretMessage := "secret message"

In [ ]:
// The GenerateKey method takes in a reader that returns random bits, and
// the number of bits
privateKey, err := rsa.GenerateKey(rand.Reader, 2048)
if err != nil {
	panic(err)
}

// The public key is a part of the *rsa.PrivateKey struct
publicKey := privateKey.PublicKey

In [ ]:
fmt.Println(secretMessage)

In [ ]:
encryptedBytes, err := rsa.EncryptOAEP(
	sha256.New(),
	rand.Reader,
	&publicKey,
	[]byte("super secret message"),
	nil)
if err != nil {
	panic(err)
}

fmt.Println("encrypted bytes: ", encryptedBytes)

In [ ]:
encryptedBytes, err := rsa.EncryptOAEP(
	sha256.New(),
	rand.Reader,
	&privateKey,
	[]byte("super secret message"),
	nil)
if err != nil {
	panic(err)
}

fmt.Println("encrypted bytes: ", encryptedBytes)

In [ ]:
import "crypto/x509"

In [ ]:
privateKeyPkcs1 := x509.MarshalPKCS1PrivateKey(privateKey)

In [ ]:
import "encoding/pem"

In [ ]:
privateKeyPem := pem.EncodeToMemory(
    &pem.Block{
        Type: "RSA PUBLIC KEY",
        Bytes: privateKeyPkcs1,
    },
)

In [ ]:
publicKeyPkcs1 := x509.MarshalPKCS1PublicKey(&privateKey.PublicKey)

In [ ]:
publicKeyPem := pem.EncodeToMemory(
    &pem.Block{
        Type: "RSA PRIVATE KEY",
        Bytes: publicKeyPkcs1,
    },
)

In [ ]:
string(publicKeyPem)

In [ ]:
string(privateKeyPem) + string(publicKeyPem)

In [ ]:
import "bytes"
pk := &rsa.PrivateKey{)
buf := bytes.NewBuffer(privateKeyPkcs1)
rsa.DecryptPKCS1v15(buf, pk, 15)

In [ ]:
x509.ParsePKCS1PublicKey(privateKeyPkcs1)

In [ ]:
privateKey.D

In [ ]:
import "crypto"
data, sig := rsa.SignPKCS1v15(nil, privateKey, crypto.Hash(0), []byte("hello world")) 
data
string(data)

In [ ]:
// Unsign verifies the message using a rsa-sha256 signature
// func (r *rsaPublicKey) Unsign(message []byte, sig []byte) error {
// 	h := sha256.New()
// 	h.Write(message)
// 	d := h.Sum(nil)
// 	return rsa.VerifyPKCS1v15(r.PublicKey, crypto.SHA256, d, sig)
// }
rsa.VerifyPKCS1v15(&privateKey.PublicKey, crypto.Hash(0), data, nil)

In [ ]:
// crypto/rand.Reader is a good source of entropy for blinding the RSA
// operation.
import "os"
rng := rand.Reader

message := []byte("message to be signed")

// Only small messages can be signed directly; thus the hash of a
// message, rather than the message itself, is signed. This requires
// that the hash function be collision resistant. SHA-256 is the
// least-strong hash function that should be used for this at the time
// of writing (2016).

signature, err := rsa.SignPKCS1v15(nil, privateKey, crypto.Hash(0), message)
if err != nil {
    fmt.Fprintf(os.Stderr, "Error from signing: %s\n", err)
}

fmt.Printf("Signature: %x\n", signature)



In [ ]:
hasher := crypto.Hash(0)
hasher.HashFunc()()
err := rsa.VerifyPKCS1v15(&privateKey.PublicKey, crypto.SHA256, []byte("message to be signed"), signature)
if err != nil {
    fmt.Fprintf(os.Stderr, "Error from verification: %s\n", err)
}

// signature is a valid signature of message from the public key.